# Model Analysis

In the previous chapter, we introduced the basic use of `PyTorch` and use a neural network model to predict the probabilities of listed firms' tax non-compliance behaviors. Our previous model may be too simple, this leads to a bad prediction performance. By tunning the hyperparameters or regularizaition, we can further improve our model.

So in this chapter, we would first introduce some regularization methods including $\ell_2$ regularization, dropout, early stopping and batch normalization. Then, we apply these methods to the example of tax non-compliant firms classfication.

**Learning Goal**:
1. Implementing regularization methods including L2 penalty, dropout, early stopping;
2. Implementing batch normalization;
3. Tunning hyperparameters.

## Contents

## 0 - Import Packages and Data

Before we start, please copy and paste `FNN` class, `train` and `test` function in last chapter to a .py file, and put the file in your current working diretory.

In [ ]:
import pandas as pd # used for data import
import numpy as np # used for numerical operations
import torch # used for tensor operations
import torch.nn as nn # used for building neural networks
from torch.utils.data import DataLoader, TensorDataset # used for creating data loaders
from sklearn.preprocessing import StandardScaler # used for standardizing features
from sklearn.metrics import average_precision_score # used for evaluating models
import matplotlib.pyplot as plt # used for plotting PR curves
from sklearn.model_selection import train_test_split # used for splitting data into train/valiadtion sets

from utils import FNN, train, test 
import itertools
import json # used for write output to txt

np.random.seed(42) # set random seed for reproducibility
torch.manual_seed(42)

Then we import data, all the procedures are the same as last chapter. For simplicity, we define a function `prepare_loader()` to transform ndarray data into DataLoader.

In [2]:
train_df = pd.read_csv('../全连接神经网络/train_data.csv') # import training data
test_df = pd.read_csv('../全连接神经网络/test_data.csv')

train_df, valid_df = train_test_split(train_df, 
                                      stratify=train_df['noncompliance'], 
                                      test_size=0.2, 
                                      random_state=42,
                                      shuffle=True)

# standardize the input
scaler = StandardScaler()
X_train_array = scaler.fit_transform(train_df.drop(['noncompliance'], axis=1))
X_valid_array = scaler.transform(valid_df.drop(['noncompliance'], axis=1))
X_test_array = scaler.transform(test_df.drop(['noncompliance'], axis=1))
y_train_array = train_df['noncompliance'].values
y_valid_array = valid_df['noncompliance'].values
y_test_array = test_df['noncompliance'].values

def prepare_loader(X_array, y_array, batch_size, shuffle):
    """
    transform array data into DataLoader
    params:
        X_array: ndarray, feature;
        y_array: ndarray, label;
        batch_size: int, batch size;
        shuffle: bool, change the order of samples;
    return:
        loader: DataLoader, data including features and labels used in pytorch.
    """
    X_tensor = torch.tensor(X_array, dtype=torch.float32)
    y_tensor = torch.tensor(y_array, dtype=torch.float32)
    dataset = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

# prepare training, validation and test data
train_loader = prepare_loader(X_train_array, y_train_array, batch_size=128, shuffle=True)
valid_loader = prepare_loader(X_valid_array, y_valid_array, batch_size=128, shuffle=False)
test_loader = prepare_loader(X_test_array, y_test_array, batch_size=128, shuffle=False)

In the following parts, we would talk about the implementation of $\ell_2$ regularization, dropout, early stopping and batch normalization respectively. Then, we would use these methods during tunning.

## 1 - $\ell_2$ Regularization

Recall the backpropagation of $\ell_2$ penalty:
$J_2 = \frac{\lambda}{2n} \sum_{l} ||W^{[l]}||_{F}^2$ 
and the derivative is 
$\frac{\partial J_2}{\partial W^{[l]}} = \frac{\lambda}{n} W^{[l]}.$
The update step for $\ell_2$ penalty is 
$$
W^{[l] (t+1)} = W^{[l] (t)} - \text{lr} \times (\text{grad}_{W^{[l]}} + \lambda \times W^{[l] (t)}).
$$

**In the optimizer without momentum** (e.g. `torch.optim.SGD`), we can implement $\ell_2$ regularization by adding $\lambda \times W$ to the gradients, instead of actually modifying the loss function. This can be done by setting the param `weight_decay`, which allows the optimizer to update parameters in the following way: 
$$
\text{param} = \text{param} - \text{lr} \times (\text{grad} + \text{weight decay} * \text{param})
$$

**In the optimizer with momentum** (e.g. `torch.optim.Adam`), `weight_decay` and $\ell_2$ regularization are different, see [Loshchilov and Hutter (2019)](https://arxiv.org/pdf/1711.05101) for more details. The solution is to use `torch.optim.AdamW` optimizer. 

In the following codes, we 
1. assign all the weight parameters to a list `decay`, and assign all the bias parameters to  `no_decay`; 
2. set `weight_decay` to be 3e-3 for weight parameters, and 0 for bias parameters.


In [41]:
model = FNN()
criterion = nn.BCELoss()

decay, no_decay = [], []
for name, param in model.named_parameters():
    if 'bias' in name or 'bn' in name:  # exclude bias and batch norm parameters from weight decay
        no_decay.append(param)
    else: # all other parameters (weights) will have weight decay
        decay.append(param)

optimizer = torch.optim.AdamW([
    {'params': decay, 'weight_decay': 3e-3}, # set weight decay for weights
    {'params': no_decay, 'weight_decay': 0.0} # set no weight decay for bias
], lr=3e-4)

[{'params': [Parameter containing:
   tensor([[-0.0058,  0.0787,  0.0335,  ..., -0.0343,  0.1886, -0.0106],
           [-0.1383, -0.0017,  0.0264,  ..., -0.1703, -0.0990, -0.0679],
           [-0.1147,  0.1227, -0.1480,  ..., -0.0491,  0.1752, -0.1304],
           ...,
           [ 0.0693, -0.1568, -0.0350,  ...,  0.1506,  0.1808, -0.1931],
           [-0.1398, -0.1738,  0.1148,  ..., -0.0646,  0.0186, -0.1057],
           [ 0.1158, -0.1126,  0.1776,  ..., -0.0869,  0.0824, -0.0789]],
          requires_grad=True),
   Parameter containing:
   tensor([[ 0.0322, -0.0001,  0.0216,  ..., -0.0278, -0.0266, -0.0835],
           [ 0.0168,  0.0094, -0.0465,  ..., -0.1020, -0.0676, -0.0126],
           [ 0.1078,  0.0735,  0.0284,  ..., -0.0300, -0.1072, -0.1108],
           ...,
           [ 0.0405, -0.0099,  0.0761,  ...,  0.0390,  0.0831,  0.0148],
           [ 0.0255,  0.1000, -0.0486,  ..., -0.0704, -0.1136,  0.0815],
           [-0.1006, -0.0309,  0.0263,  ...,  0.0507, -0.0593, -0.0708]],

In [ ]:
train(model, train_loader, criterion, optimizer, num_epochs=500)

In [ ]:
def l2_norm_all_weights(model):
    total_norm = 0.0
    for name, param in model.named_parameters():
        if 'weight' in name and param.requires_grad:
            total_norm += torch.norm(param, p=2) ** 2
    return total_norm.sqrt()

l2_norm_all_weights(model) # check the l2 norm of all weights

In [ ]:
nn_l2_pred_probs = test(model, test_loader, criterion)
average_precision_score(y_test_array, nn_l2_pred_probs)

## 2 - Dropout

In the following codes, we build a network with dropout.

In the `__init__` method, `self.dropout1 = nn.Dropout(p=0.5)` defines a dropout layer with dropout probability of 0.5, it would randomly zeroes some elements of the input with probability $p=0.5$ during training mode.

In the `forward` method, `x = self.dropout1(x)` applies the dropout operator to the output of first layer.

In [ ]:
class FNN_dropout(nn.Module):
    """
    A fully connected neural network with dropout.
    """
    def __init__(self):
        super(FNN_dropout, self).__init__()
        self.fc1 = nn.Linear(24, 64)
        self.dropout1 = nn.Dropout(p=0.5) # define dropout layer with 50% probability
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(p=0.3) # define dropout layer with 30% probability
        self.fc3 = nn.Linear(32, 8)
        self.dropout3 = nn.Dropout(p=0.1) # define dropout layer with 10% probability
        self.fc4 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x) # apply dropout
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x) # apply dropout
        x = torch.relu(self.fc3(x))
        x = self.dropout3(x) # apply dropout
        x = torch.sigmoid(self.fc4(x))
        return x

In [ ]:
model_dropout = FNN_dropout()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_dropout.parameters(), lr=3e-4)
train(model_dropout, train_loader, criterion, optimizer, num_epochs=500)

In [ ]:
nn_dropout_pred_probs = test(model_dropout, test_loader, criterion)
average_precision_score(y_test_array, nn_dropout_pred_probs) # 0.3123840521753748

## 3 - Early Stopping

The implementation of early stopping is easy, we just need to modify the `train` function, if no significant improvement for some epochs, then we break the loop.

In the following codes, we define a `EarlyStopping` class to detect significant validation loss improvement. `__call__()` method would:
1. if significant improvement detected, then save the model and update `best_valid_loss`
2. if no significant improvement detected, then `counter += 1`
   1. if `counter` >= `patience`, then break the detect.

In [11]:
class EarlyStopping:
    '''Early stops training if validation loss doesnt imporve after patience'''
    def __init__(self, patience=50, delta=0, path='checkpoint.pt', verbose=False):
        '''
        params:
            patience: int, number of epochs to wait since no improvement
            delta: float, loss reduction required to detect siginificant improvement
            path: path to save the checkpoint
        '''
        self.patience = patience
        self.delta = delta
        self.counter = 0 # counter of patience
        self.stop = False # flag for stop training
        self.best_valid_loss = None
        # self.verbose = verbose
        self.path = path # path to save the check point
    
    def __call__(self, valid_loss, model):
        if self.best_valid_loss is None:
            self.best_valid_loss = valid_loss
            self.save_checkpoint(valid_loss, model)
        elif valid_loss < self.best_valid_loss - self.delta:
            # significant improvement detect
            self.best_valid_loss = valid_loss
            self.save_checkpoint(valid_loss, model)
            self.counter = 0
        else:
            # no significant improvement
            self.counter += 1
            if self.counter >= self.patience:
                self.stop = True

    def save_checkpoint(self, valid_loss, model):
        '''save models when validation loss decreases'''
        # if self.verbose:
        #     print(f'loss improvement: {valid_loss:.6f}. Saving model.')
        torch.save(model.state_dict(), self.path)

After defining the `EarlyStopping` class, we define a `train_and_validate` function in the following way
1. train the model, then return a trained model (nearly the same as last chapter's `train` function);
2. validate on the validation set, and return `valid_loss`;
3. call the `early_stopping` object to detect significant improment, if `early_stopping.early_stop == True`, then we break the loop. 

In [8]:
def train_and_validate(model, train_loader, valid_loader, criterion, optimizer, num_epochs, use_early_stop=True):
    '''
    train the model and early stopping on the validation loss
    params:
        train_loader: DataLoader, training data;
        valid_loader: DataLoader, validation data;
        criterion: loss function;
        optimizer: optimizer that update parameters
        num_epochs: int, number of epochs
        use_early_stop: bool, if False then train until the last epoch
    return:
        model: trained model
        train_losses: list, training loss of all epoches
        valid_losses: list, validation loss of all epoches
    '''
    train_losses, valid_losses = [], [] # used to store loss of all epochs
    early_stopping = EarlyStopping() # initialize an EarlyStopping object

    for epoch in range(num_epochs):
        # train the model
        model.train()
        acc_train_loss = 0 # used to store training loss of all batches
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad() # clear previous gradients
            output = model(data) # forward propagation
            loss = criterion(output.squeeze(1), target) # calculate average batch loss
            loss.backward() # backpropagation
            optimizer.step() # update parameters
            acc_train_loss += loss.item() * data.size(0) # accumulate loss
        train_loss = acc_train_loss / len(train_loader.dataset) # calculate average epoch loss
        train_losses.append(train_loss)
        if (epoch + 1) % 50 == 0: # print loss each 50 epochs
            print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {train_loss:.4f}")

        # validate the model
        valid_loss, _ = test(model, valid_loader, criterion)
        valid_losses.append(valid_loss)

        # early stopping
        if use_early_stop:
            early_stopping(valid_loss, model)
            if early_stopping.stop:
                print(f'early stopping in epoch {epoch}.')
                break
            model.load_state_dict(torch.load('checkpoint.pt', weights_only=True)) # load the best model
    return model, train_losses, valid_losses
    

In the following codes, we train the model with early stopping. The training procedures stops at epoch 142, and epoch 92 has the minimum validation loss.

In [ ]:
model = FNN()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
model, train_losses, valid_losses = train_and_validate(model, train_loader, valid_loader, criterion, optimizer, 500, use_early_stop=False)

We can plot the training loss and validation loss of all epochs.

In [ ]:
plt.plot(range(len(train_losses)),train_losses, label='Training Loss')
plt.plot(range(len(valid_losses)),valid_losses,label='Validation Loss')
plt.axvline(valid_losses.index(min(valid_losses)), linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.xlim(0, len(train_losses))
plt.grid(True)
plt.legend()
plt.show()

## 4 - Batch Normalization

In the following codes, we build a fully connected neural network with batch normalization.

In the `__init__` method, `self.bn1 = nn.BatchNorm1d(64)` defines a batch normalization over a 2D input with 64 features.

In the `forward` method, `x = torch.relu(self.bn1(self.fc1(x)))` firstly do the linear transformation, then apply batch normalization, and then do nonlinear activation.

In [ ]:
class FNN_bn(nn.Module):
    """
    A fully connnected neural network with batch normalization
    """
    def __init__(self):
        super(FNN_bn, self).__init__()
        self.fc1 = nn.Linear(24, 64)
        self.bn1 = nn.BatchNorm1d(64) # defines batch normalization over a 2D input with 64 features
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32) # defines batch normalization over a 2D input with 32 features
        self.fc3 = nn.Linear(32, 8)
        self.bn3 = nn.BatchNorm1d(8) # defines batch normalization over a 2D input with 8 features
        self.fc4 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.sigmoid(self.fc4(x))
        return x

In [ ]:
model_bn = FNN_bn()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_bn.parameters(), lr=3e-4)
train(model_bn, train_loader, criterion, optimizer, num_epochs=500)

In [ ]:
nn_bn_pred_probs = test(model_bn, test_loader, criterion)
average_precision_score(y_test_array, nn_bn_pred_probs)

## 5 - Tunning

In [3]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_layers, dropout_rates, use_batchnorm=False):
        '''
        params:
            input_dim: int, dimension of input features;
            hidden_layers: list, number of neurons for each hidden layers;
            dropout_rates: list, same length as hidden_layers, dropout probability for each hidden layer;
            use_batchnorm: bool, use batch normalization in each layer.
        '''
        super(Net, self).__init__()
        layers = [] 
        prev_dim = input_dim
        for  i, hidden_dim in enumerate(hidden_layers):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            if use_batchnorm:
                layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            if dropout_rates[i] > 0:
                layers.append(nn.Dropout(dropout_rates[i]))
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, 1))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

`param_grid` includes all the possible values of hyperparameters. Since `hidden_layers` and `dropout_rates` must have the same length, after expanding the grid, we need to filter out those combinations with different lengths.

In [38]:
param_grid = {
    'batch_size': [128],
    'hidden_layers': [[64, 32, 16], [32, 8]],
    'dropout_rates': [[0.5, 0.3, 0.1], [0.3, 0.2]],
    'batch_norm': [True, False],
    'criterion': ['BCE'],
    'optimizer': ['Adam'],
    'lr': [1e-3, 3e-4],
    'weight_decay': [0, 1e-4],
    'num_epochs': [100, 300, 500],
    'early_stop': [True, False],
}

# expand the grid to all combinations
keys, values = zip(*param_grid.items())
all_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# filter out combination that hidden_layers and dropout_rates have different lengths
all_combinations = [
    combo for combo in all_combinations 
    if len(combo['hidden_layers']) == len(combo['dropout_rates'])
]

In [ ]:
def tune(combinations, output_file='output.txt'):
    with open(output_file, 'w') as f:
        pass

    for config in combinations:
        print(f'config: {config}')
    
        # prepare data
        train_loader = prepare_loader(X_train_array, y_train_array, batch_size=config['batch_size'], shuffle=True)
        valid_loader = prepare_loader(X_valid_array, y_valid_array, batch_size=config['batch_size'], shuffle=False)
        
        # build model
        model = Net(
            input_dim=next(iter(train_loader))[0].shape[1],
            hidden_layers=config['hidden_layers'],
            dropout_rates=config['dropout_rates'],
            use_batchnorm=config['batch_norm']
        )
        
        # loss
        if config['criterion'] == 'BCE':
            criterion = nn.BCELoss()
        else:
            raise ValueError('Unknown criterion')
        
        # optimizer
        decay, no_decay = [], []
        for name, param in model.named_parameters():
            if 'bias' in name or 'bn' in name:
                no_decay.append(param)
            else:
                decay.append(param)

        if config['optimizer'] == 'Adam':
            optimizer = torch.optim.Adam([
                {'params': decay, 'weight_decay': config['weight_decay']},
                {'params': no_decay, 'weight_decay': 0.0}
            ], lr=3e-4)
        else:
            raise ValueError("Unknown optimizer")
        
        # train & validate
        model, _, _ = train_and_validate(model, train_loader, valid_loader, criterion, optimizer, config['num_epochs'], config['early_stop'])
        
        # save results
        valid_loss, pred_probs = test(model, valid_loader, criterion)
        config['valid_loss'] = valid_loss
        valid_auprc = average_precision_score(y_valid_array, pred_probs)
        result = {**config, "valid_auprc": valid_auprc}

        with open(output_file, 'a') as f:
            f.write(json.dumps(result) + '\n')

In [40]:
tune(all_combinations)

config: {'batch_size': 128, 'hidden_layers': [64, 32, 16], 'dropout_rates': [0.5, 0.3, 0.1], 'batch_norm': True, 'criterion': 'BCE', 'optimizer': 'Adam', 'lr': 0.001, 'weight_decay': 0, 'num_epochs': 100, 'early_stop': True}
Epoch [50/100] Loss: 0.1740
Epoch [100/100] Loss: 0.1727
config: {'batch_size': 128, 'hidden_layers': [64, 32, 16], 'dropout_rates': [0.5, 0.3, 0.1], 'batch_norm': True, 'criterion': 'BCE', 'optimizer': 'Adam', 'lr': 0.001, 'weight_decay': 0, 'num_epochs': 100, 'early_stop': False}
Epoch [50/100] Loss: 0.1676
Epoch [100/100] Loss: 0.1654
config: {'batch_size': 128, 'hidden_layers': [64, 32, 16], 'dropout_rates': [0.5, 0.3, 0.1], 'batch_norm': True, 'criterion': 'BCE', 'optimizer': 'Adam', 'lr': 0.001, 'weight_decay': 0, 'num_epochs': 300, 'early_stop': True}
Epoch [50/300] Loss: 0.1784
Epoch [100/300] Loss: 0.1763
Epoch [150/300] Loss: 0.1726
Epoch [200/300] Loss: 0.1725
Epoch [250/300] Loss: 0.1707
early stopping in epoch 255.
config: {'batch_size': 128, 'hidden_l

KeyboardInterrupt: 

NameError: name 'results' is not defined